In [1]:
data_var = '2023-12-04'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1762,2023-12-04,Brasil Nbb,19:30,Brasília,São José,3.12,1.33,159.5,1.86,1.84,5.5,2.07,1.67,hEwwzWM6,0.320513,0.751880,0.537634,0.543478,0.072393,0.0,0.0,NaN,NaN,249.818,217.142469,0.869203,166.858,84.946344,0.509094,0.000,0.000,177.39,106.86,0.0,0.0,0.0,0.0,0.568863,0.007644,0.151253,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1763,2023-12-04,Brasil Nbb,20:00,R10 Score Vasco da Gama,Franca,3.20,1.32,152.5,1.85,1.85,5.5,2.09,1.66,0vwsZkxD,0.312500,0.757576,0.540541,0.540541,0.070076,0.0,0.0,NaN,NaN,130.384,45.797464,0.351251,119.534,33.184865,0.277619,0.000,0.000,113.90,132.66,0.0,0.0,0.0,0.0,0.588213,0.000000,0.162163,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1764,2023-12-04,Brasil Nbb,20:00,São Paulo,Botafogo,1.08,6.84,159.5,1.80,1.86,-15.5,2.07,1.67,4OwW8k8D,0.925926,0.146199,0.555556,0.537634,0.072125,0.0,0.0,NaN,NaN,123.810,22.728807,0.183578,272.000,147.297978,0.541537,0.000,0.000,161.25,300.24,0.0,0.0,0.0,0.0,1.028519,0.023184,0.151253,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1765,2023-12-04,Argentina Liga A,20:00,Platense,San Lorenzo,1.55,2.34,145.5,1.88,1.79,-4.5,1.94,1.77,hIPB4664,0.645161,0.427350,0.531915,0.558659,0.072512,0.0,0.0,NaN,NaN,156.488,62.111430,0.396909,138.744,30.763840,0.221731,0.000,0.000,91.64,183.26,0.0,0.0,0.0,0.0,0.287205,0.034681,0.064802,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1766,2023-12-04,Argentina Liga A,21:10,Obras,Regatas,1.33,3.12,158.5,1.84,1.86,-8.5,2.09,1.66,j5X9mPgt,0.751880,0.320513,0.543478,0.537634,0.072393,0.0,0.0,NaN,NaN,179.336,66.307174,0.369737,115.980,16.676049,0.143784,0.000,0.000,130.56,104.94,0.0,0.0,0.0,0.0,0.568863,0.007644,0.162163,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1767,2023-12-04,Ásia Wasl,13:30,Al Manama,Al Shamal,1.01,13.00,164.5,1.85,1.85,-20.5,2.00,1.72,6uQ0dhim,0.990099,0.076923,0.540541,0.540541,0.067022,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,124.32,0.00,0.0,0.0,0.0,0.0,1.210308,0.000000,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1768,2023-12-04,Bulgária Nbl,14:00,Rilski Sportist,Yambol,1.17,4.61,160.5,1.87,1.79,-13.5,2.10,1.65,SWuUAB9a,0.854701,0.216920,0.534759,0.558659,0.071621,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,95.59,390.61,0.0,0.0,0.0,0.0,0.841677,0.030912,0.169706,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1769,2023-12-04,China Cba,08:35,Beijing Royal Fighters,Beijing,1.65,2.19,202.5,1.84,1.90,-4.5,2.06,1.66,YVZEkUW6,0.606061,0.456621,0.543478,0.526316,0.062682,0.8,0.2,NaN,NaN,248.522,126.898722,0.510614,182.016,99.697404,0.547740,226.082,163.498,460.52,110.00,1.0,0.0,0.0,0.0,0.198874,0.022688,0.152066,-2.11,-0.422,-1.540284,0.0,0.0,0.0,-2.62,-0.524,-2.270992,0.0,0.0,0.0
1770,2023-12-04,China Cba,08:35,Guangdong,Fujian,1.12,5.85,219.5,1.87,1.87,-15.5,1.95,1.75,0xS0DXOD,0.892857,0.170940,0.534759,0.534759,0.063797,0.0,0.8,NaN,NaN,126.018,23.224581,0.184296,266.632,142.039528,0.532717,0.000,358.410,129.47,422.24,0.0,0.0,0.0,0.0,0.959717,0.000000,0.076444,0.00,0.000,inf,0.0,0.0,0.0,-2.31,-0.462,-10.497835,0.0,0.0,0.0
1771,2023-12-04,China Cba,08:35,Shanxi Zhongyu,Sichuan,1.09,6.69,221.5,1.80,1.92,-16.5,1.97,1.74,vTQ4CivK,0.917431,0.149477,0.555556,0.520833,0.066908,0.4,0.8,NaN,NaN,237.514,79.733460,0.335700,447.012,170.172573,0.380689,223.486,725.866,135.00,705.00,0.0,1.0,0.0,0.0,1.017943,0.045620,0.087674,0.05,0.010,9.000000,0.0,0.0,0.0,-5.00,-1.000,-5.690000

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
18,21:30,Eua Torneio Nba In,Indiana Pacers,Boston Celtics,239.5,1.89,1.0,Over
